In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import zfit
from zfit import z

In [ ]:
import mplhep as hep
hep.style.use(hep.style.CMS)
plt.rcParams.update({
    "text.usetex": True,
})

In [ ]:
from plotting_meta.plotting_meta import bins

from bff_plotting_tools.make_hists import make_sys, SysHist
from bff_plotting_tools.data_getter import get_data

from bff_signal_model.bff_bck_model import bff_bck_model, sigma, theta, mean

In [ ]:
era = '2016'
df, lumi = get_data(era)
df = df[(df.DiLepMass > bins.calc_bin_range()[0]) & (df.DiLepMass <  bins.calc_bin_range()[1])]
compute_hesse=True
binning_type = 'split'

In [ ]:
obs = zfit.Space("x", limits=bins.calc_bin_range())

In [ ]:
def fit_lognorm(y, weights, obs, verbose=False):
    lognorm = bff_bck_model(obs, sigma=sigma, theta=theta, mean=mean)

    # load the data
    data = zfit.Data.from_numpy(obs=obs, array=y, weights=weights)
    
    # build the loss
    nll = zfit.loss.UnbinnedNLL(model=lognorm, data=data)

    # minimize
    minimizer = zfit.minimize.Minuit()
    result = minimizer.minimize(nll)
    lognorm.result = result
    
    # calculate errors
    param_errors = result.hesse()
    
    if verbose: print(result)
    
    return lognorm, data

In [ ]:
def fit_region(reg):
    # get data
    region = df[(df.type=='data') & (df['{}_nom'.format(reg)]==1)]
    #make data_hist
    y = region.DiLepMass.to_numpy()
    weight = region.Weight.to_numpy()
    hist = make_sys(region, 'DiLepMass', reg)
    # fit data
    ln, data = fit_lognorm(y, weight, obs)
    # make fit hist
    normalization = np.sum(hist.nominal)
    fit = ln.fill_bins(bins, normalization, supersample=1)
    return {"hist":hist, "fit": fit}

In [ ]:
plot_dict = {}
for reg in ['CR10', 'CR13', 'CR14', 'CR20', 'CR23', 'CR24']:
    print(reg)
    plot_dict[reg] = fit_region(reg)

In [ ]:
for reg, reg_dict in plot_dict.items():
    fit, hist = reg_dict['fit'], reg_dict['hist']
    fig, ax = plt.subplots(figsize=(10,10))
    hist.make_density_hist().draw(ax, color='red', label='Data')
    fit.make_density_hist().draw(ax, color='green', label='Fit')
    ax.set_yscale('log')
    ax.set_ylabel('Events per GeV')
    ax.set_xlabel('DiLepMass [GeV]')
    ax.set_ylim(bottom=1e-2, top=1e4)
    ax.legend(title = '{}'.format(reg))
    fig.savefig('fits/abcd/{}_binning_density_{}_{}.png'.format(binning_type, era, reg))

In [ ]:
#make abcd:
#A*B/C = D
for nJets in [1,2]:
    reg = 'SR{}'.format(nJets)
    def ufd(reg): return plot_dict[reg]['fit'].uncertainty_std_dev()
    A,B,C = ufd('CR{}0'.format(nJets)), ufd('CR{}3'.format(nJets)), ufd('CR{}4'.format(nJets))
    D = A*B/C
    ABCD = SysHist.from_ufloats(bins, D)
    
    fig, ax = plt.subplots(figsize=(10,10))
    ABCD.make_density_hist().draw(ax, color='red', label='ABCD')
    ax.set_yscale('log')
    ax.set_ylabel('Events per GeV')
    ax.set_xlabel('DiLepMass [GeV]')
    ax.set_ylim(bottom=1e-2, top=1e4)
    ax.legend(title = '{}'.format(reg))
    fig.savefig('fits/abcd/{}_binning_density_{}_{}.png'.format(binning_type, era, reg))